# Week 8 Lab: Predator-Prey Dynamics

## The Lotka-Volterra Model

**SCIE1500 - Analytical Methods for Scientists**

---

### Learning Objectives

By the end of this lab, you will be able to:

1. Write and interpret the Lotka-Volterra predator-prey equations
2. Calculate predator efficiency from model parameters
3. Find equilibrium (fixed) points by setting derivatives to zero
4. Verify whether a given point is an equilibrium
5. Solve the model numerically using `odeint`
6. Create phase portraits with vector fields and trajectories
7. Connect model behavior to real ecological systems

---

### What to Submit

1. **During Lab:** Complete **Exercise E** and show your results to your lab demonstrator
2. **By Due Date:** Upload screenshots of completed **Exercises E, F, and G** including the phase diagram generated

---

### Terminology Note

The following terms are used interchangeably:
- **Fixed points** = **Equilibrium points** = **Steady-state solutions**
- **Vector field** = **Slope field** = **Direction field**
- **Phase portrait** = **Phase diagram**

---

In [ ]:
# === SETUP: Run this cell first ===
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.integrate import odeint

# For nicer symbolic display
sp.init_printing(use_unicode=True)

print("✓ All packages loaded successfully!")
print("\nThis week: Predator-Prey Dynamics — When Populations Interact")

---

## Part A: The Lotka-Volterra Model

### A.1 The Model Equations

The **Lotka-Volterra model** (1920s) describes how predator and prey populations interact over time.

Let:
- $H$ = prey (Herbivore) population
- $P$ = predator population

The model is a system of two **coupled ordinary differential equations (ODEs)**:

$$\boxed{\frac{dH}{dt} = \alpha H - \beta HP}$$

$$\boxed{\frac{dP}{dt} = \lambda HP - \gamma P}$$

### A.2 Parameter Interpretation

| Parameter | Symbol | Meaning | Effect |
|-----------|--------|---------|--------|
| Prey birth rate | $\alpha$ | Natural reproduction rate of prey | + for prey |
| Predation rate | $\beta$ | Rate of prey kills per predator | − for prey |
| Predator efficiency | $\lambda$ | Predator births per prey consumed | + for predator |
| Predator death rate | $\gamma$ | Natural death rate of predators | − for predator |

### A.3 Understanding Each Term

**Prey equation:** $\frac{dH}{dt} = \underbrace{\alpha H}_{\text{births}} - \underbrace{\beta HP}_{\text{deaths from predation}}$

**Predator equation:** $\frac{dP}{dt} = \underbrace{\lambda HP}_{\text{births from feeding}} - \underbrace{\gamma P}_{\text{natural deaths}}$

The term $HP$ represents **mass action**: more encounters (and predation) when both populations are large.

### A.4 Setting Up the Parameters

In [ ]:
# Define parameters (these are the exam-style values)
alpha = 0.2      # prey birth rate
beta = 0.005     # predation rate on prey
lambda_ = 0.001  # predator birth rate from feeding (note: lambda is reserved in Python)
gamma = 0.6      # predator death rate

# Store as tuple for passing to functions
params = (alpha, beta, lambda_, gamma)

print("Model Parameters:")
print(f"  α (prey birth rate)     = {alpha}")
print(f"  β (predation rate)      = {beta}")
print(f"  λ (predator efficiency) = {lambda_}")
print(f"  γ (predator death rate) = {gamma}")

---

## Part B: Special Cases — What Happens Without the Other Species?

### B.1 Prey Without Predators ($P = 0$)

If there are no predators, the prey equation becomes:
$$\frac{dH}{dt} = \alpha H$$

This is **exponential growth** with solution $H(t) = H_0 e^{\alpha t}$

In [ ]:
# Prey without predator: exponential growth
t = np.linspace(0, 20, 100)
H0 = 1000  # Initial prey population
H_t = H0 * np.exp(alpha * t)

plt.figure(figsize=(8, 4))
plt.plot(t, H_t, 'b-', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Prey Population (H)')
plt.title(f'Prey Without Predators: Exponential Growth at Rate α = {alpha}')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Without predators, prey grow from {H0} to {H_t[-1]:.0f} in 20 time units")

### B.2 Predators Without Prey ($H = 0$)

If there is no prey (food), the predator equation becomes:
$$\frac{dP}{dt} = -\gamma P$$

This is **exponential decay** with solution $P(t) = P_0 e^{-\gamma t}$

In [ ]:
# Predator without prey: exponential decay (starvation)
t = np.linspace(0, 20, 100)
P0 = 400  # Initial predator population
P_t = P0 * np.exp(-gamma * t)

plt.figure(figsize=(8, 4))
plt.plot(t, P_t, 'r-', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Predator Population (P)')
plt.title(f'Predators Without Prey: Exponential Decay at Rate γ = {gamma}')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Without prey, predators decline from {P0} to {P_t[-1]:.2f} in 20 time units")

---

## Part C: Predator Efficiency

**Predator efficiency** measures how effectively predators convert consumed prey into new predators:

$$\boxed{\epsilon = \frac{\lambda}{\beta}}$$

**Interpretation:**
- $\beta HP$ = prey killed per unit time
- $\lambda HP$ = predators born per unit time
- $\epsilon$ = predators born per prey killed

In [ ]:
# Calculate predator efficiency
efficiency = lambda_ / beta

print(f"Predator Efficiency Calculation:")
print(f"  ε = λ/β = {lambda_}/{beta} = {efficiency}")
print(f"\nInterpretation: {efficiency*100:.0f}% efficiency")
print(f"  → For every {int(1/efficiency)} prey consumed, {1} new predator is born")

---

## Part D: Finding Equilibrium Points

### D.1 What is an Equilibrium?

An **equilibrium** (or **fixed point**) is a state where the system doesn't change over time:

$$\frac{dH}{dt} = 0 \quad \text{AND} \quad \frac{dP}{dt} = 0$$

### D.2 Finding Equilibria Algebraically

**From prey equation:** $\frac{dH}{dt} = H(\alpha - \beta P) = 0$
- Either $H = 0$ (extinction) OR $P = \frac{\alpha}{\beta}$

**From predator equation:** $\frac{dP}{dt} = P(\lambda H - \gamma) = 0$
- Either $P = 0$ (extinction) OR $H = \frac{\gamma}{\lambda}$

This gives us **two equilibria**:

1. **Extinction equilibrium:** $(H^*, P^*) = (0, 0)$

2. **Coexistence equilibrium:** $(H^*, P^*) = \left(\frac{\gamma}{\lambda}, \frac{\alpha}{\beta}\right)$

In [ ]:
# Find equilibrium points symbolically
H_sym, P_sym = sp.symbols('H P', positive=True)
alpha_s, beta_s, lambda_s, gamma_s = sp.symbols('alpha beta lambda gamma', positive=True)

# Define the ODEs
dHdt_sym = alpha_s * H_sym - beta_s * H_sym * P_sym
dPdt_sym = lambda_s * H_sym * P_sym - gamma_s * P_sym

# Solve for equilibria
equilibria = sp.solve([dHdt_sym, dPdt_sym], [H_sym, P_sym])
print("Equilibrium points (symbolic):")
for eq in equilibria:
    print(f"  (H*, P*) = {eq}")

In [ ]:
# Calculate numerical equilibrium for our parameters
H_star = gamma / lambda_
P_star = alpha / beta

print(f"\nNumerical equilibria for α={alpha}, β={beta}, λ={lambda_}, γ={gamma}:")
print(f"  1. Extinction equilibrium: (H*, P*) = (0, 0)")
print(f"  2. Coexistence equilibrium: (H*, P*) = ({H_star}, {P_star})")

### D.3 Verifying an Equilibrium

To verify a point is an equilibrium, substitute into BOTH equations and check both give zero.

In [ ]:
# Verify the coexistence equilibrium (600, 40)
H_test, P_test = H_star, P_star

dHdt_test = alpha * H_test - beta * H_test * P_test
dPdt_test = lambda_ * H_test * P_test - gamma * P_test

print(f"Verification of equilibrium at (H, P) = ({H_test}, {P_test}):")
print(f"  dH/dt = {alpha}×{H_test} - {beta}×{H_test}×{P_test} = {dHdt_test}")
print(f"  dP/dt = {lambda_}×{H_test}×{P_test} - {gamma}×{P_test} = {dPdt_test}")

if dHdt_test == 0 and dPdt_test == 0:
    print("\n✓ Both derivatives are zero — this IS an equilibrium!")
else:
    print("\n✗ Not an equilibrium")

---

## Part E: Defining the ODE System for Numerical Solution

To solve the model numerically, we need a function that `odeint` can use.

In [ ]:
def lotka_volterra(populations, t, params):
    """
    Defines the Lotka-Volterra ODEs for odeint.
    
    Parameters:
    -----------
    populations : array-like
        Current state [H, P] (prey, predator)
    t : float
        Current time (required by odeint, but not used in L-V)
    params : tuple
        (alpha, beta, lambda_, gamma)
    
    Returns:
    --------
    list : [dH/dt, dP/dt]
    """
    H, P = populations
    alpha, beta, lambda_, gamma = params
    
    # Prey equation: dH/dt = αH - βHP
    dHdt = alpha * H - beta * H * P
    
    # Predator equation: dP/dt = λHP - γP
    dPdt = lambda_ * H * P - gamma * P
    
    return [dHdt, dPdt]

# Test at equilibrium (should return [0, 0])
test_result = lotka_volterra([H_star, P_star], 0, params)
print(f"Test at equilibrium ({H_star}, {P_star}):")
print(f"  dH/dt = {test_result[0]}, dP/dt = {test_result[1]}")

# Test away from equilibrium
test_result2 = lotka_volterra([700, 10], 0, params)
print(f"\nTest at (700, 10):")
print(f"  dH/dt = {test_result2[0]:.2f} (prey {'increasing' if test_result2[0] > 0 else 'decreasing'})")
print(f"  dP/dt = {test_result2[1]:.2f} (predator {'increasing' if test_result2[1] > 0 else 'decreasing'})")

### Determining Population Trends at Different Points

Let's check the four regions of the phase plane.

In [ ]:
# Check population trends at four different points
test_points = [
    (700, 10, "high prey, low predator"),
    (700, 100, "high prey, high predator"),
    (300, 10, "low prey, low predator"),
    (300, 100, "low prey, high predator")
]

print(f"Population trends (equilibrium at H*={H_star}, P*={P_star}):")
print("="*60)

for H, P, description in test_points:
    rates = lotka_volterra([H, P], 0, params)
    H_trend = "↑ increasing" if rates[0] > 0 else "↓ decreasing"
    P_trend = "↑ increasing" if rates[1] > 0 else "↓ decreasing"
    
    print(f"\n({H}, {P}) — {description}:")
    print(f"  Prey (H):     {H_trend} (dH/dt = {rates[0]:.1f})")
    print(f"  Predator (P): {P_trend} (dP/dt = {rates[1]:.1f})")

---

## Part F: Solving the Model with `odeint`

The `odeint` function numerically integrates the ODEs to find population trajectories over time.

In [ ]:
# Time array: 0 to 100 with 500 points
t = np.linspace(0, 100, 500)

# Initial conditions: start away from equilibrium
H0, P0 = 700, 10  # More prey, fewer predators than equilibrium
initial_state = [H0, P0]

# Solve the ODEs
# IMPORTANT: args=(params,) must have a comma to make it a tuple!
solution = odeint(lotka_volterra, initial_state, t, args=(params,))

# Extract H(t) and P(t)
H_t = solution[:, 0]  # First column = prey
P_t = solution[:, 1]  # Second column = predator

print(f"Solution computed!")
print(f"  Time range: 0 to {t[-1]}")
print(f"  Initial: H = {H_t[0]:.1f}, P = {P_t[0]:.1f}")
print(f"  Final:   H = {H_t[-1]:.1f}, P = {P_t[-1]:.1f}")

### F.1 Time Series Plot

In [ ]:
# Plot both populations over time
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(t, H_t, 'b-', linewidth=2, label='Prey (H)')
ax.plot(t, P_t, 'r-', linewidth=2, label='Predator (P)')

# Add equilibrium reference lines
ax.axhline(y=H_star, color='b', linestyle='--', alpha=0.5, label=f'H* = {H_star}')
ax.axhline(y=P_star, color='r', linestyle='--', alpha=0.5, label=f'P* = {P_star}')

ax.set_xlabel('Time', fontsize=12)
ax.set_ylabel('Population', fontsize=12)
ax.set_title('Lotka-Volterra Time Series: Predator-Prey Oscillations', fontsize=14)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Notice: Predator peaks LAG behind prey peaks (predator response is delayed)")

---

## Part G: Phase Portraits

A **phase portrait** shows trajectories in the $(H, P)$ space, eliminating time to show the relationship between populations directly.

### G.1 Basic Phase Portrait

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Plot trajectory in phase space
ax.plot(H_t, P_t, 'b-', linewidth=1.5, label='Trajectory')

# Mark initial point
ax.plot(H0, P0, 'go', markersize=12, label=f'Start ({H0}, {P0})')

# Mark equilibrium
ax.plot(H_star, P_star, 'r*', markersize=15, label=f'Equilibrium ({H_star}, {P_star})')
ax.plot(0, 0, 'k*', markersize=10, label='Origin (0, 0)')

# Add nullclines
ax.axhline(y=P_star, color='orange', linestyle=':', alpha=0.7, label=f'H-nullcline: P = {P_star}')
ax.axvline(x=H_star, color='purple', linestyle=':', alpha=0.7, label=f'P-nullcline: H = {H_star}')

ax.set_xlabel('Prey (H)', fontsize=12)
ax.set_ylabel('Predator (P)', fontsize=12)
ax.set_title('Phase Portrait: Lotka-Volterra Model', fontsize=14)
ax.legend(loc='upper right')
ax.set_xlim(0, 1600)
ax.set_ylim(0, 200)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("The trajectory forms a CLOSED ORBIT around the equilibrium.")
print("This represents sustained periodic oscillations.")

### G.2 Adding Direction Arrows

Arrows show the direction of time evolution (which way the system moves).

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Plot trajectory
ax.plot(H_t, P_t, 'b-', linewidth=1.5)

# Add arrow to show direction
# Key: Choose a time index where the trajectory is visible
arrow_index = 50  # Adjust this to place arrow where desired

# Get position and direction at that time
H_arrow = H_t[arrow_index]
P_arrow = P_t[arrow_index]
dH = H_t[arrow_index + 1] - H_t[arrow_index]  # Change in H
dP = P_t[arrow_index + 1] - P_t[arrow_index]  # Change in P

# Scale factor for arrow visibility
scale = 15

ax.annotate('', 
            xy=(H_arrow + scale*dH, P_arrow + scale*dP),  # Arrow tip
            xytext=(H_arrow, P_arrow),  # Arrow base
            arrowprops=dict(arrowstyle='->', color='red', lw=2))

# Mark points
ax.plot(H0, P0, 'go', markersize=10, label='Start')
ax.plot(H_star, P_star, 'k*', markersize=15, label=f'Equilibrium ({H_star}, {P_star})')

ax.set_xlabel('Prey (H)', fontsize=12)
ax.set_ylabel('Predator (P)', fontsize=12)
ax.set_title('Phase Portrait with Direction Arrow', fontsize=14)
ax.legend()
ax.set_xlim(0, 1500)
ax.set_ylim(0, 150)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Arrow placed at position ({H_arrow:.1f}, {P_arrow:.1f})")
print("Flow is COUNTERCLOCKWISE around the equilibrium!")

### G.3 Building a Vector Field

A vector field shows the direction of flow at many points across the phase plane.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Create grid of points
H_range = np.arange(50, 1501, 100)   # Prey values
P_range = np.arange(5, 151, 10)      # Predator values

# Draw vector field
for H in H_range:
    for P in P_range:
        # Compute derivatives at this point
        dHdt = alpha * H - beta * H * P
        dPdt = lambda_ * H * P - gamma * P
        
        # Skip if derivatives are both near zero (equilibrium)
        magnitude = np.sqrt(dHdt**2 + dPdt**2)
        if magnitude < 0.1:
            continue
        
        # Normalize for consistent segment length
        dH_norm = dHdt / magnitude
        dP_norm = dPdt / magnitude
        
        # Scale for visibility
        scale = 30
        
        # Draw slope line
        ax.plot([H - scale*dH_norm, H + scale*dH_norm],
                [P - scale*dP_norm, P + scale*dP_norm],
                'gray', linewidth=0.5, alpha=0.7)

# Mark equilibrium
ax.plot(H_star, P_star, 'r*', markersize=15, zorder=5, label=f'Equilibrium ({H_star}, {P_star})')

# Add nullclines
ax.axhline(y=P_star, color='blue', linestyle='--', alpha=0.5, label=f'H-nullcline: P = {P_star}')
ax.axvline(x=H_star, color='green', linestyle='--', alpha=0.5, label=f'P-nullcline: H = {H_star}')

ax.set_xlabel('Prey (H)', fontsize=12)
ax.set_ylabel('Predator (P)', fontsize=12)
ax.set_title('Direction Field for Lotka-Volterra Model', fontsize=14)
ax.legend()
ax.set_xlim(0, 1600)
ax.set_ylim(0, 160)
ax.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

---

## 📝 STUDENT EXERCISE E (Show Demonstrator)

### Find the Equilibrium Points

For the Lotka-Volterra model with parameters:
- $\alpha = 0.2$
- $\beta = 0.005$
- $\gamma = 0.6$
- $\lambda = 0.001$

**Tasks:**

1. Calculate the coexistence equilibrium $(H^*, P^*)$ using the formulas:
   - $H^* = \gamma / \lambda$
   - $P^* = \alpha / \beta$

2. Verify that this point is an equilibrium by substituting into both ODEs.

3. Calculate the predator efficiency $\epsilon = \lambda / \beta$.

In [ ]:
# EXERCISE E: Complete the calculations below

# Given parameters
alpha_E = 0.2
beta_E = 0.005
gamma_E = 0.6
lambda_E = 0.001

# Task 1: Calculate equilibrium
# H_star_E = gamma_E / lambda_E
# P_star_E = alpha_E / beta_E
# print(f"Coexistence equilibrium: (H*, P*) = ({H_star_E}, {P_star_E})")

# Task 2: Verify by substitution
# dHdt_check = alpha_E * H_star_E - beta_E * H_star_E * P_star_E
# dPdt_check = lambda_E * H_star_E * P_star_E - gamma_E * P_star_E
# print(f"\nVerification: dH/dt = {dHdt_check}, dP/dt = {dPdt_check}")

# Task 3: Calculate efficiency
# efficiency_E = lambda_E / beta_E
# print(f"\nPredator efficiency: ε = {efficiency_E} = {efficiency_E*100}%")

---

## 📝 STUDENT EXERCISE F (Upload)

### Complete Phase Portrait with Multiple Trajectories

Create a comprehensive phase portrait that includes:

1. A vector field (in light gray or another light color)
2. Trajectories from **6 different initial conditions**:
   - (700, 10)
   - (250, 50)
   - (800, 50)
   - (1250, 100)
   - (1250, 200)
   - (300, 100)
3. Direction arrows on each trajectory
4. The equilibrium point marked
5. Appropriate labels, title, and legend

In [ ]:
# EXERCISE F: Complete Phase Portrait

# Set up figure
# fig, ax = plt.subplots(figsize=(12, 9))

# 1. Draw vector field (light gray background)
# H_range = np.arange(50, 1501, 75)
# P_range = np.arange(5, 201, 10)
# ...

# 2. Solve for multiple initial conditions
# initial_conditions = [
#     (700, 10, 'blue'),
#     (250, 50, 'green'),
#     (800, 50, 'purple'),
#     (1250, 100, 'orange'),
#     (1250, 200, 'brown'),
#     (300, 100, 'cyan')
# ]
#
# t = np.linspace(0, 100, 500)
#
# for H0, P0, color in initial_conditions:
#     sol = odeint(lotka_volterra, [H0, P0], t, args=(params,))
#     ax.plot(sol[:, 0], sol[:, 1], color=color, linewidth=1.5)
#     ax.plot(H0, P0, 'o', color=color, markersize=8)  # Start point
#     
#     # Add arrow at time index 40
#     idx = 40
#     dH = sol[idx+1, 0] - sol[idx, 0]
#     dP = sol[idx+1, 1] - sol[idx, 1]
#     ax.annotate('', xy=(sol[idx, 0] + 10*dH, sol[idx, 1] + 10*dP),
#                 xytext=(sol[idx, 0], sol[idx, 1]),
#                 arrowprops=dict(arrowstyle='->', color=color, lw=1.5))

# 3. Mark equilibria
# ax.plot(H_star, P_star, 'r*', markersize=20, zorder=10)

# 4. Add labels, title, grid
# ...
# plt.show()

---

## 📝 STUDENT EXERCISE G (Upload)

### Predator-Prey Practice Quiz Answers

Complete the **Predator-prey practice quiz (Week 8)** on LMS, then record your answers below.

Double-click this cell to edit, type your answers, then press Shift+Enter to render.

**EXERCISE G: Practice Quiz Answers**

Q1. Answer: _________________

Q2. Answer: _________________

Q3. Answer: _________________

Q4. Answer: _________________

Q5. Answer: _________________

---

## Part H: Extension — Model with Carrying Capacity

A more realistic model includes **carrying capacity** for prey:

$$\frac{dH}{dt} = \alpha H\left(1 - \frac{H}{K}\right) - \beta HP$$

This changes the behavior dramatically: orbits become **spirals** that converge to equilibrium.

In [ ]:
# Modified model with carrying capacity
def lotka_volterra_K(populations, t, params):
    """L-V model with carrying capacity for prey."""
    H, P = populations
    alpha, beta, lambda_, gamma, K = params
    
    # Modified prey equation with logistic growth
    dHdt = alpha * H * (1 - H/K) - beta * H * P
    dPdt = lambda_ * H * P - gamma * P
    
    return [dHdt, dPdt]

# Parameters with carrying capacity
K = 1500
params_K = (alpha, beta, lambda_, gamma, K)

# New equilibrium with K
H_star_K = gamma / lambda_
P_star_K = (alpha / beta) * (1 - H_star_K / K)

print(f"Basic model equilibrium: ({H_star}, {P_star})")
print(f"Modified model equilibrium: ({H_star_K}, {P_star_K:.1f})")

# Compare trajectories
t = np.linspace(0, 150, 1000)
initial = [150, 50]

sol_basic = odeint(lotka_volterra, initial, t, args=(params,))
sol_mod = odeint(lotka_volterra_K, initial, t, args=(params_K,))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(sol_basic[:,0], sol_basic[:,1], 'b-')
axes[0].plot(H_star, P_star, 'r*', markersize=15)
axes[0].set_title('Basic Lotka-Volterra\n(Closed Orbits)', fontsize=12)
axes[0].set_xlabel('Prey (H)')
axes[0].set_ylabel('Predator (P)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(sol_mod[:,0], sol_mod[:,1], 'r-')
axes[1].plot(H_star_K, P_star_K, 'r*', markersize=15)
axes[1].set_title(f'With Carrying Capacity K={K}\n(Spiral to Equilibrium)', fontsize=12)
axes[1].set_xlabel('Prey (H)')
axes[1].set_ylabel('Predator (P)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey insight: Adding carrying capacity creates DAMPED oscillations")
print("that converge to a STABLE equilibrium — more realistic!")

---

## Summary: Key Formulas

| Concept | Formula |
|---------|--------|
| Prey equation | $\frac{dH}{dt} = \alpha H - \beta HP$ |
| Predator equation | $\frac{dP}{dt} = \lambda HP - \gamma P$ |
| Predator efficiency | $\epsilon = \frac{\lambda}{\beta}$ |
| Extinction equilibrium | $(H^*, P^*) = (0, 0)$ |
| Coexistence equilibrium | $(H^*, P^*) = \left(\frac{\gamma}{\lambda}, \frac{\alpha}{\beta}\right)$ |
| Prey increasing when | $P < \frac{\alpha}{\beta}$ |
| Predator increasing when | $H > \frac{\gamma}{\lambda}$ |

### Key Python Patterns

```python
# Define ODE function
def lotka_volterra(populations, t, params):
    H, P = populations
    # ... compute derivatives ...
    return [dHdt, dPdt]

# Solve with odeint
solution = odeint(lotka_volterra, initial_state, t, args=(params,))
# IMPORTANT: args=(params,) needs the comma!

# Extract solutions
H_t = solution[:, 0]  # prey
P_t = solution[:, 1]  # predator
```

### Exam Tips (Q34)

1. **Equilibrium formulas:** $H^* = \gamma/\lambda$, $P^* = \alpha/\beta$ — note the "cross" pattern
2. **Efficiency:** $\epsilon = \lambda/\beta$ (not $\beta/\lambda$!)
3. **Verification:** Substitute into BOTH equations, both must give zero
4. **Direction of flow:** Always counterclockwise in basic L-V
5. **With carrying capacity:** Closed orbits become spirals to equilibrium

---

## What's Next?

**Week 9** shifts to **Probability and Uncertainty**:
- How do we quantify the chance of events?
- What is Bayes' theorem?
- How do diagnostic tests work?

---

*The Lotka-Volterra model shows how simple mathematical rules can produce complex, oscillating behavior — a fundamental insight in population ecology!*